### __LIBRARIES IMPORTATION__

In [ ]:
import fastf1 as ff1
from fastf1 import plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.collections import LineCollection
import fastf1 as ff1
import numpy as np
import pandas as pd
import pickle
import os

### __GENERAL DATA__

In [ ]:
year = 2023
circuit = 'Bahrein'
test_number = 1
session = 1

#### __STEP 0:__ Data Generation and pkl creation

In [ ]:
# ff1.Cache.enable_cache('C:/Users/migue/Documents/F1 Data Center/Datos/fastf1_Cache')
# test = ff1.get_testing_session(2023,1,3)
# test.load()

#### __STEP 1:__ Data Transformation into Pandas DF

In [ ]:
path = f'C:\\Users\\migue\\Documents\\F1 Data Center\\Datos\\fastf1_Testing_Timing_Data\\{year}\\timing_data_t{test_number}_s{session}.ff1pkl'
with open(path, 'rb') as f:
    data = pickle.load(f)

dataTuple = data['data'][0]

df = pd.DataFrame(dataTuple)

#Remove Rows with more than 2 Nan Values
nanDF = df.isna()
df['TotalNan'] = nanDF.sum(axis=1) 

df = df.loc[(df['TotalNan'] <= 2), :]
df = df.drop(['TotalNan'], axis=1)
df = df.reset_index(drop=True)


#BUGS
df = df.loc[df['Driver'].astype(int) < 100, :]

print(df.shape)

colNames: list = df.columns.values.tolist() #Column names
print(colNames)

df.head()

#### __STEP 2:__ Preprocessing

In [ ]:
df = df.drop(['PitOutTime', 'PitInTime', 'Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','IsPersonalBest'], axis=1)

In [ ]:
import sys
path = "/Users/migue/Documents/F1 Data Center/F1-Data-Center/Programs/Python/"
sys.path.append(path)
import multidata

# Transform Driver Number into Drivers Name
def transformNum(row, year = year):
    numb = row['Driver'] 
    return multidata.numberToNames(year, numb).name

df['Driver'] = df.apply(transformNum, axis=1)

#Transform time instances into miliseconds except 'Time' which will be transformed into minutes

def toMins(row):
    try: 
        return row['Time'].total_seconds() / 60
    except:
        return np.nan

def toMiliscs(elem):
    try: 
        return elem.total_seconds() *1000
    except:
        return np.nan

df['Time'] = df.apply(toMins, axis=1)
df['LapTime'] = df['LapTime'].apply(toMiliscs)
df['Sector1Time'] = df['Sector1Time'].apply(toMiliscs)
df['Sector2Time'] = df['Sector2Time'].apply(toMiliscs)
df['Sector3Time'] = df['Sector3Time'].apply(toMiliscs)
display(df)

TESTING TIME 

In [ ]:
def minSecDec(n: int):
    if not pd.isna(n):
        minutes = int(n//60000)
        n = n%60000
        sec = int(n//1000)
        dec =  int(n%1000)
        if sec < 10:
            sec = '0'+str(sec)
        if dec < 10:
            dec = '00'+str(dec)
        elif dec < 100:
            dec = '0'+str(dec)
        string = f'{minutes}:{sec}.{dec}'
        return string
    else: return n

driv = 'Sergio PEREZ'
dfDriv = df.loc[(df['Driver'] == driv), :]
allTimes = dfDriv['LapTime'].to_numpy()
new_allTimes = np.empty_like(allTimes,  dtype='object')
for i,elem in enumerate(allTimes):
    new_allTimes[i] = minSecDec(elem)
print(new_allTimes)

#### Split in Morning/Afternoon

In [ ]:
dfMorn = df.loc[(df['Time'] <= 250), :]
dfAfter = df.loc[(df['Time'] > 250), :]

In [ ]:
times = df['LapTime'].to_numpy()
# minTime = np.nanmin(times)
def returnQualy_RaceLaps(times: np.array, mode: str = 'tt', minTime = 0):
    qualyLaps = []
    qualyIndeces = []
    raceLaps = []
    raceIndices = []
    minTime = np.nanmin(times)
    qualyTop = (minTime * 103)/100
    raceTop = (minTime * 110)/100
    for t,time in enumerate(times):
        if time != np.nan:
            if time <= qualyTop:
                qualyLaps.append(time)
                qualyIndeces.append(t)
            elif time <= raceTop:
                raceLaps.append(time)
                raceIndices.append(t)

    if mode == 'tt': return np.array(qualyLaps), qualyIndeces, np.array(raceLaps), raceIndices
    else: return np.array(qualyLaps), np.array(raceLaps)



In [ ]:
drivers = ['Max VERSTAPPEN', 'Carlos SAINZ', 'George RUSSELL', 'Felipe DRUGOVICH', 'ZHOU Guanyu']
# drivers = dfMorn['Driver'].unique()
colors  = multidata.f1_teams(drivers, year, 'Default', 'Default').color
for d,driv in enumerate(drivers):
    dfDriv = dfMorn.loc[(df['Driver'] == driv), :]
    allTimes = dfDriv['LapTime'].to_numpy()
    qualyLaps, qualyIndeces, raceLaps, raceIndices = returnQualy_RaceLaps(allTimes)
    plt.plot(qualyLaps[:10], linewidth=2 , color = colors[d], label = driv) 

ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('TOP 5 - QUALY PACE FIRST 5 LAPS-MORNING') 
plt.xlabel("Laps")
plt.ylabel("Time")
plt.show()



In [ ]:
for d,driv in enumerate(drivers):
    dfDriv = dfMorn.loc[(df['Driver'] == driv), :]
    allTimes = dfDriv['LapTime'].to_numpy()
    qualyLaps, qualyIndeces, raceLaps, raceIndices = returnQualy_RaceLaps(allTimes)
    plt.plot(raceLaps[:35], linewidth=2 , color = colors[d], label = driv) 

ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('TOP 5 - RACE PACE FIRST 35 LAPS-MORNING') 
plt.xlabel("Laps")
plt.ylabel("Time")
plt.show()

In [ ]:
drivers = ['Max VERSTAPPEN', 'Charles LECLERC', 'Lewis HAMILTON', 'Fernando ALONSO', 'Valtteri BOTTAS']
# drivers = dfAfter['Driver'].unique()
colors  = multidata.f1_teams(drivers, year, 'Default', 'Default').color
for d,driv in enumerate(drivers):
    dfDriv = dfAfter.loc[(df['Driver'] == driv), :]
    allTimes = dfDriv['LapTime'].to_numpy()
    qualyLaps, qualyIndeces, raceLaps, raceIndices = returnQualy_RaceLaps(allTimes)
    plt.plot(qualyLaps[:10], linewidth=2 , color = colors[d], label = driv) 

ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('TOP 5 - QUALY PACE FIRST 5 LAPS-AFTERNOON') 
plt.xlabel("Laps")
plt.ylabel("Time")
plt.show()

In [ ]:
for d,driv in enumerate(drivers):
    dfDriv = dfAfter.loc[(df['Driver'] == driv), :]
    allTimes = dfDriv['LapTime'].to_numpy()
    qualyLaps, qualyIndeces, raceLaps, raceIndices = returnQualy_RaceLaps(allTimes)
    plt.plot(raceLaps[:40], linewidth=2 , color = colors[d], label = driv) 

ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('TOP 5 - RACE PACE FIRST 35 LAPS-AFTERNOON') 
plt.xlabel("Laps")
plt.ylabel("Time")
plt.show()